In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import math
import numpy as np
import pandas as pd

from preprocessing import Data_Sat, Norm, add_delta_time
from model_builder import LSTM, load_model

data_folder = './data/'
file_name = 'score_80_v1.model'
path = data_folder  + file_name

In [8]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device('cuda:0')
    from torch.cuda import FloatTensor
else:
    device = torch.device('cpu')
    from torch import FloatTensor

In [9]:
# data settings
sequence_length = 5
max_sequence_count = 50

# train settings
batch_size = 10
epoch_count = 10
plot_draw = False

# optimizer settings
learning_rate = 1e-3
weight_decay = 0

# model settings
lstm_hidden_dim = 10
lstm_hidden_lauers_count = 1
bidirectional = False
dropout = 0

# scheduler settings
factor = 0.1
patience = 2
threshold = 1e-2

model = LSTM(device, lstm_hidden_dim=lstm_hidden_dim,
             lstm_layers_count=lstm_hidden_lauers_count,
             bidirectional=bidirectional,
             dropout=dropout,
            ).to(device)

loss_function = torch.nn.MSELoss()
optimizer = optim.Adam(
                        model.parameters(),
                        lr=learning_rate, 
                        weight_decay=weight_decay
                    )

scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=factor, 
                              patience=patience, verbose=True, threshold=threshold
                              )

In [10]:
train_hist, val_hist = [], []

In [11]:
load_model(path, model, optimizer, scheduler, train_hist, val_hist)

successfully loaded


In [16]:
test = pd.read_csv(data_folder + "/Track 1/test.csv", parse_dates=['epoch'])
columns = ['id', 'sat_id', 'delta_seconds', 'x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
test_with_dt = add_delta_time(test, columns)
test_with_dt.set_index(keys='sat_id', drop=False, inplace=True)

AttributeError: Can only use .dt accessor with datetimelike values

In [13]:
test

,id,sat_id,epoch,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,3927,1,2014-02-01 00:01:45.162,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774
1,3928,1,2014-02-01 00:22:57.007,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402
2,3929,1,2014-02-01 00:44:08.852,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412
3,3930,1,2014-02-01 01:05:20.697,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600
4,3931,1,2014-02-01 01:26:32.542,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724
...,...,...,...,...,...,...,...,...,...
284066,1231060,597,2014-02-28 05:19:01.386,28595.031634,-85458.623976,5021.767767,-1.721131,-0.011611,0.122357
284067,1231061,597,2014-02-28 07:21:46.454,15547.173728,-84233.509948,5840.616702,-1.815323,0.353445,0.098927
284068,1231062,597,2014-02-28 09:24:31.522,1981.107111,-80123.860051,6458.394351,-1.858963,0.773846,0.067379
284069,1231063,597,2014-02-28 11:27:16.590,-11644.801379,-72674.504171,6805.361999,-1.826030,1.262930,0.024782
